# Import Libraries

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
import tensorflow as tf
from sklearn.metrics import r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Bidirectional, GRU, Conv1D, MaxPooling1D, Flatten, Dropout

# Load Dataset

In [ ]:
data = pd.read_csv("2w.csv")

# Extract features and target variable

In [ ]:
features = data[['CF(PCU/min)', 'CF*2(PCU/min)', '%_of_4W', '%_of_4W*2']].values
target = data['SD'].values

# Normalize data

In [ ]:
scaler = MinMaxScaler()
features = scaler.fit_transform(features)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

In [ ]:
X_train_lstm = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test_lstm = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Load the saved models

In [ ]:
def create_bilstm_model():
    model = Sequential()
    model.add(Bidirectional(LSTM(128, activation='relu', input_shape=(X_train_lstm.shape[1], 1), return_sequences=True)))
    model.add(Bidirectional(LSTM(64, activation='relu', return_sequences=False)))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model

bilstm_model = create_bilstm_model()
bilstm_model.fit(X_train_lstm, y_train, epochs=600, batch_size=32, validation_data=(X_test_lstm, y_test), verbose=1)


In [ ]:
def create_bigru_model():
    model = Sequential()
    model.add(Bidirectional(GRU(128, activation='relu', input_shape=(X_train_lstm.shape[1], 1), return_sequences=True)))
    model.add(Bidirectional(GRU(48, activation='relu', return_sequences=False))) 
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model

bigru_model = create_bigru_model()
bigru_model.fit(X_train_lstm, y_train, epochs=800, batch_size=62, validation_data=(X_test_lstm, y_test), verbose=1)

In [ ]:
def create_cnn_bilstm_model():
    model = Sequential()
    model.add(Conv1D(filters=128, kernel_size=2, activation='relu', input_shape=(X_train_lstm.shape[1], 1)))
    model.add(MaxPooling1D(pool_size=1))
    model.add(Bidirectional(LSTM(50, activation='relu')))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model

cnn_bilstm_model = create_cnn_bilstm_model()
cnn_bilstm_model.fit(X_train_lstm, y_train, epochs=600, batch_size=32, validation_data=(X_test_lstm, y_test), verbose=1)

# Number of times to test

In [ ]:
num_tests = int(input("Enter the number of tests: "))

# Ask for user input and print output

In [ ]:
for _ in range(num_tests):
    cf_pcu_min = float(input("Enter CF(PCU/min): "))
    cf_2_pcu_min = float(input("Enter CF*2(PCU/min): "))
    percent_4w = float(input("Enter %_of_4W: "))
    percent_4w_2 = float(input("Enter %_of_4W*2: "))
    user_input = np.array([[cf_pcu_min, cf_2_pcu_min, percent_4w, percent_4w_2]])
    
    user_input = scaler.transform(user_input)

    user_input_lstm = user_input.reshape((user_input.shape[0], user_input.shape[1], 1))

    prediction_bilstm = bilstm_model.predict(user_input_lstm)[0][0]

    prediction_bigru = bigru_model.predict(user_input_lstm)[0][0]

    prediction_cnn_bilstm = cnn_bilstm_model.predict(user_input_lstm)[0][0]

    print("\nService Delay Predictions:")
    print(f"BiLSTM Prediction: {prediction_bilstm:.4f} seconds")
    print(f"BiGRU Prediction: {prediction_bigru:.4f} seconds")
    print(f"CNN BiLSTM Prediction: {prediction_cnn_bilstm:.4f} seconds")